In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
import matplotlib.pyplot as plt 
from torchtext.data import Field, TabularDataset, BucketIterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import seaborn as sns

c:\users\baps\anaconda3\envs\lstm\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
#We will check the shape of the data frame
data.shape

(20800, 5)

In [4]:
# check how many values are none and we have to drop it.
data.isnull().sum(axis=0)

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = data.dropna()

In [6]:
df.shape

(18285, 5)

In [7]:
#we have deleted all the data which had NAN values becasue it can affect the accuracy.

In [8]:
# get the independent and dependent variables
x = df.drop('label',axis=1)
y = df['label']

In [9]:
x.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [12]:
#take a vocabulary size 
vocab_size = 5000

# One hot representation of data

In [13]:
# As we dropped the values we have to reset the index to make one hot representation of data
sentences = x.copy()
sentences.reset_index(inplace=True)

In [14]:
# Now, we have to preprocess the text because we have many ounctuations and STOPWORDS in our dataset so we have to preprocess that data
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baps\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
#here we can see that corpus contains the words after preprocessing done.
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [18]:
# We can see that the words in corpus are represented in onehot encode w.r.t. given vocab_size that is 5000
onehot = [one_hot(words,vocab_size) for words in corpus]
onehot[:10]

[[1076, 2004, 3855, 3964, 1251, 1177, 2432, 4548, 4821, 3157],
 [3090, 3921, 277, 3803, 561, 2494, 2349],
 [792, 2085, 3099, 206],
 [1083, 2836, 2939, 3433, 2700, 2344],
 [4308, 561, 750, 666, 2017, 2368, 561, 415, 869, 208],
 [4623,
  3529,
  4621,
  2659,
  3924,
  4115,
  2845,
  2475,
  4603,
  4988,
  1575,
  959,
  92,
  1630,
  2349],
 [4348, 1735, 189, 352, 3582, 757, 60, 4393, 373, 1561, 684],
 [1554, 61, 1640, 1548, 2048, 1673, 4115, 2910, 373, 1561, 684],
 [2484, 2659, 3720, 3690, 509, 4227, 4554, 310, 4115, 1168],
 [653, 1048, 646, 2146, 2026, 1062, 3558, 4097]]

In [19]:
len(onehot)

18285

# Embedding of onehot words

In [20]:
length_df = pd.DataFrame(corpus,columns=['corpus'])

In [21]:
length_df['length'] = length_df['corpus'].str.len()

In [22]:
length_df['length'].max()

299

In [23]:
#Here we have maximum sentence length is 299 so we will take embedding accordingly

In [24]:
length = 30
embedding = pad_sequences(onehot,maxlen=length,padding='pre')
embedding[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 1076, 2004,
        3855, 3964, 1251, 1177, 2432, 4548, 4821, 3157],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 3090, 3921,  277, 3803,  561, 2494, 2349],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  792, 2085, 3099,  206],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1083, 2836, 2939, 3433, 2700, 2344],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 4308,  561,
         750,  666, 2017, 2368, 

In [25]:
# Create a model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
X = np.array(embedding)
Y = np.array(y)

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

In [28]:
#train the model 
model.fit(X_train,y_train,batch_size=64,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
192/192 [==============================] - 9s 45ms/step - loss: 0.3227 - accuracy: 0.8448 - val_loss: 0.2034 - val_accuracy: 0.9152
Epoch 2/20
192/192 [==============================] - 8s 39ms/step - loss: 0.1433 - accuracy: 0.9411 - val_loss: 0.1821 - val_accuracy: 0.9238
Epoch 3/20
192/192 [==============================] - 8s 44ms/step - loss: 0.0897 - accuracy: 0.9668 - val_loss: 0.2037 - val_accuracy: 0.9218
Epoch 4/20
192/192 [==============================] - 10s 54ms/step - loss: 0.0573 - accuracy: 0.9803 - val_loss: 0.2556 - val_accuracy: 0.9193
Epoch 5/20
192/192 [==============================] - 10s 52ms/step - loss: 0.0307 - accuracy: 0.9903 - val_loss: 0.3273 - val_accuracy: 0.9158
Epoch 6/20
192/192 [==============================] - 9s 49ms/step - loss: 0.0170 - accuracy: 0.9956 - val_loss: 0.3483 - val_accuracy: 0.9168
Epoch 7/20
192/192 [==============================] - 9s 47ms/step - loss: 0.0138 - accuracy: 0.9950 - val_loss: 0.4654 - val_accuracy: 0.90

In [29]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features,input_length=length))
model1.add(Dropout(0.3))
model1.add(Bidirectional(LSTM(200)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [30]:
#train the model 
model1.fit(X_train,y_train,batch_size=64,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
192/192 [==============================] - 18s 94ms/step - loss: 0.3229 - accuracy: 0.8443 - val_loss: 0.2085 - val_accuracy: 0.9026
Epoch 2/20
192/192 [==============================] - 21s 108ms/step - loss: 0.1551 - accuracy: 0.9378 - val_loss: 0.1851 - val_accuracy: 0.9234
Epoch 3/20
192/192 [==============================] - 22s 117ms/step - loss: 0.1096 - accuracy: 0.9584 - val_loss: 0.2011 - val_accuracy: 0.9195
Epoch 4/20
192/192 [==============================] - 23s 118ms/step - loss: 0.0726 - accuracy: 0.9740 - val_loss: 0.2118 - val_accuracy: 0.9213
Epoch 5/20
192/192 [==============================] - 25s 132ms/step - loss: 0.0455 - accuracy: 0.9841 - val_loss: 0.2560 - val_accuracy: 0.9163
Epoch 6/20
192/192 [==============================] - 23s 121ms/step - loss: 0.0356 - accuracy: 0.9884 - val_loss: 0.3061 - val_accuracy: 0.9158
Epoch 7/20
192/192 [==============================] - 26s 133ms/step - loss: 0.0266 - accuracy: 0.9909 - val_loss: 0.3575 - val_acc

We can see that after adding dropout we are not getting accuracy more than 90% so we will take different approaches but we can see the entire process of how LSTM cab ne used in text classification

# Now can check the performance of the model 

In [31]:
y_pred=model.predict_classes(X_test)
CM = confusion_matrix(y_test,y_pred)
score = accuracy_score(y_test,y_pred)
print(CM)
print(score)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[3122  297]
 [ 250 2366]]
0.9093620546810274
